In [35]:
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

In [79]:
df=pd.read_excel("Stopwords_Removed_Ticket_Type.xlsx",sheet_name='Sheet2')

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
Incident: Number      40 non-null int64
Opened Date           40 non-null datetime64[ns]
Queue                 40 non-null object
Category              40 non-null object
Client Location       38 non-null object
Template_Name         21 non-null object
Request Definition    16 non-null object
Description           40 non-null object
Ticket Type           40 non-null object
Description1          40 non-null object
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 3.2+ KB


In [81]:
df=df.dropna(subset=['Description1'])

In [6]:
pipeline = Pipeline([
        ('bow', CountVectorizer(ngram_range=(1,2))),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        #('Normalize', preprocessing.Normalizer()),
        ('c',LinearSVC()),  
    ])

In [82]:
pipeline = Pipeline([
        ('bow', CountVectorizer(ngram_range=(1,2))),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        ('Normalize', preprocessing.Normalizer()),
        ('c',svm.SVC(kernel='linear',probability=True)),  
    ])

In [ ]:
pipeline = Pipeline([
        ('bow', CountVectorizer(ngram_range=(1,2))),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        ('Normalize', preprocessing.Normalizer()),
        ('c',naive_bayes.MultinomialNB(fit_prior=True)),  
    ])

In [90]:
X_train, X_test, y_train, y_test =  train_test_split(df['Description1'], df['Queue'], test_size=0.5, random_state=42)

In [91]:
clf=SVC(kernel='linear')

In [104]:
fit=pipeline.fit(X_train,y_train)

In [105]:
pred=pipeline.predict(X_test)
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("Accuracy : ",accuracy_score(pred,y_test))

[[0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 7 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]]
                  precision    recall  f1-score   support

EntComms-Network       0.00      0.00      0.00         1
   EntComms-O365       0.00      0.00      0.00         1
      FAC_HQ1011       0.00      0.00      0.00         1
    Hosting-Unix       0.00      0.00      0.00         1
    IT-AV-Events       0.80      1.00      0.89         4
  L2-Servicedesk       0.00      0.00      0.00         2
       euro-pool       0.00      0.00      0.00         1
     global-pool       0.47      1.00      0.64         7
    hr-EMEA-pool       0.00      0.00      0.00         1
          vcadit       0.00      0.00      0.00         1

       micro avg       0.55      0.55      0.55        20
       macro avg       0.13      0.20      0.15        20
    weighted 

C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [107]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['linear']} 

In [100]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)

In [108]:

parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}
gs_clf_svm = GridSearchCV(pipeline, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


ValueError: Invalid parameter vectorizer for estimator Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_...ar', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]). Check the list of available parameters with `estimator.get_params().keys()`.

NameError: name 'estimator' is not defined

In [70]:
grid.fit(X_train,y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................


C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt t

ValueError: could not convert string to float: 'petah tikva israel global cadence guest issue israel week nobody non company cadence guests cannot path authentication process via cadence guest portal error auth server timeout assign wlan'

In [ ]:
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

# Loading the Digits dataset
X = df['Description1']
y = df['Queue']

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()